In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input

In [2]:
data= pd.read_csv(r"C:\Users\hsynt\Downloads\Selcuk Abi\Selcuk Abi\VeriSeti2.csv")

In [3]:
# Özellik ve Hedef Ayrımı
X = data.drop(columns=['Target_10', 'adjclose'])
y = data['Target_10']

# Zaman Serisi Formatına Getirme
def create_sequences(data, target, seq_length):
    sequences, targets = [], []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i + seq_length].flatten())  # Flatten sequences for MLP
        targets.append(target[i + seq_length])
    return np.array(sequences), np.array(targets)

SEQ_LENGTH = 10
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_seq, y_seq = create_sequences(X_scaled, y.values, SEQ_LENGTH)

# Eğitim ve test kümelerinin oluşturulması
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42, stratify=y_seq)

# PCA ve LDA Veri Hazırlığı
pca = PCA(n_components=10)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

lda = LDA(n_components=1)
X_train_lda = lda.fit_transform(X_train, y_train)
X_test_lda = lda.transform(X_test)

# Performans Metriklerini Hesaplama
def calculate_metrics(y_true, y_pred, y_pred_proba):
    return {
        "Accuracy": accuracy_score(y_true, y_pred) * 100,
        "Precision": precision_score(y_true, y_pred) * 100,
        "Recall": recall_score(y_true, y_pred) * 100,
        "F1-Score": f1_score(y_true, y_pred) * 100,
        "AUC": roc_auc_score(y_true, y_pred_proba) * 100
    }

# MLP Modeli Tanımlama
def build_mlp_model(input_shape):
    model = Sequential([
        Input(shape=(input_shape,)),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Performansları Değerlendirme
results_train = []
results_test = []

for reduction, X_tr, X_te, y_tr, y_te in zip(
    ['None', 'PCA', 'LDA'],
    [X_train, X_train_pca, X_train_lda],
    [X_test, X_test_pca, X_test_lda],
    [y_train, y_train, y_train],  # Aynı y_train değerlerini kullanıyoruz
    [y_test, y_test, y_test]      # Aynı y_test değerlerini kullanıyoruz
):
    mlp_model = build_mlp_model(X_tr.shape[1])
    mlp_model.fit(X_tr, y_tr, epochs=10, batch_size=32, verbose=1)

    # Eğitim Performansı
    y_train_pred = (mlp_model.predict(X_tr) > 0.5).astype("int32").flatten()
    y_train_proba = mlp_model.predict(X_tr).flatten()
    train_metrics = calculate_metrics(y_tr, y_train_pred, y_train_proba)
    train_metrics['Reduction'] = reduction
    train_metrics['Model'] = 'MLP'
    results_train.append(train_metrics)

    # Test Performansı
    y_test_pred = (mlp_model.predict(X_te) > 0.5).astype("int32").flatten()
    y_test_proba = mlp_model.predict(X_te).flatten()
    test_metrics = calculate_metrics(y_te, y_test_pred, y_test_proba)
    test_metrics['Reduction'] = reduction
    test_metrics['Model'] = 'MLP'
    results_test.append(test_metrics)

# Eğitim ve Test Sonuçlarını Görselleştirme
train_df = pd.DataFrame(results_train).set_index(["Reduction", "Model"])
test_df = pd.DataFrame(results_test).set_index(["Reduction", "Model"])

# Eğitim Sonuçları Tablosu
print("Eğitim Kümesi Performans Sonuçları:")
print(train_df)

# Test Sonuçları Tablosu
print("\nTest Kümesi Performans Sonuçları:")
print(test_df)

# Sonuçları Kaydetme
train_df.to_csv("mlp_pca_lda_train_performance.csv")
test_df.to_csv("mlp_pca_lda_test_performance.csv")
print("\nSonuçlar 'mlp_pca_lda_train_performance.csv' ve 'mlp_pca_lda_test_performance.csv' dosyalarına kaydedildi.")

Epoch 1/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8248 - loss: 0.4158
Epoch 2/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8697 - loss: 0.3157
Epoch 3/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8737 - loss: 0.3045
Epoch 4/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8769 - loss: 0.2950
Epoch 5/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8775 - loss: 0.2911
Epoch 6/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8805 - loss: 0.2848
Epoch 7/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8777 - loss: 0.2870
Epoch 8/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8816 - loss: 0.2811
Epoch 9/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8781 - loss: 0.2847
Epoch 10/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8835 - loss: 0.2741
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 1s 601us/step
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 1s 635us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 